### Frequency Counter with Monitors (Java) [6 points]

Implement a class `FrequencyCounter` for counting the occurrences of events numbered from `0` to `max - 1`. The class has to be thread-safe, meaning that if multiple threads call `count` and `frequency` simultaneously, the effect is that of executing the calls sequentially in arbitrary order. Implement the class as a monitor so that only one thread can enter `count`, but multiple threads can call `frequency`. State the class invariant and preconditions of all methods as comments! Do not check those with assertions.

In [15]:
%%writefile frequencycounter.java
import java.util.Random;
class FrequencyCounter {
    int[] frequencies;

    //Invariant: ∀ i ε 0 .. max - 1 · frequencies[i] ≥ 0 ∧ len(frequencies) = max
    //Precondition: max > 0
    FrequencyCounter(int max) {
        // Create a frequency counter for events numbered 0 to max - 1.
        this.frequencies = new int[max];
    }

    //Precondition: 0 ≤ event < max
    void count(int event) {
        // Increment the frequency of event by 1.
        synchronized(this) {frequencies[event]++;}
    }

    //Precondition: 0 ≤ event < max
    int frequency(int event) {
        return frequencies[event];
    }
}
class Eventer extends Thread {
    FrequencyCounter fc;
    Eventer(FrequencyCounter fc) {
        this.fc = fc;
    }
    public void run() {
        Random r = new Random();
        for (int i = 0; i < 20000; i++) {
            fc.count(r.nextInt(10));
        }
    }
}
class TestFrequencyCounter {
    public static void main(String[] args) {
        final int E = 1000; // number of eventers
        FrequencyCounter fc = new FrequencyCounter(10);
        Eventer ev[] = new Eventer[E];
        for (int i = 0; i < E; i++) ev[i] = new Eventer(fc);
        long startTime = System.currentTimeMillis();
        for (int i = 0; i < E; i++) ev[i].start();
        for (int i = 0; i < E; i++) {
            try {ev[i].join();} catch (Exception e) {}
        }
        long endTime = System.currentTimeMillis();
        System.out.println(endTime - startTime);
    }
}

Overwriting frequencycounter.java


In [16]:
!javac frequencycounter.java

In [17]:
!java TestFrequencyCounter

1780


Now implement the class `HighFrequencyCounter` that behaves like `FrequencyCounter`, but is more efficient in allowing concurrent updates to different events. For example, the calls `fc.count(3)` and `fc.count(5)` should be allowed to enter the monitor and update the respective counter simultaneously. State the class invariant as a comment! Do not check the class invariant by assertions.

In [18]:
%%writefile highfrequencycounter.java
import java.util.Random;
class HighFrequencyCounter extends FrequencyCounter {
    Semaphore[] locks;
    
    class Semaphore {
    int val;
    Semaphore(int init) {
        val = init;
    }
    synchronized void P() throws InterruptedException {
        while (val == 0) wait();
        val -= 1;
    }
    synchronized void V() {
        val += 1;
        notify();
    }
    }

    //Invariant: ∀ i ε 0 .. max - 1 · frequencies[i] ≥ 0 ∧ len(frequencies) = max ∧  ∀ i ε 0 .. max - 1 · 0 ≤ locks[i] ≤ 1
    HighFrequencyCounter(int max) {
        super(max);
        this.locks = new Semaphore[max];
        for (int i = 0; i < max; i++)
            locks[i] = new Semaphore(1);
    }

    //Precondition: 0 ≤ event < max ∧ 0 ≤ locks[event] ≤ 1
    void count(int event) {
        try {locks[event].P();} catch (Exception e) {}
        frequencies[event] += 1;
        locks[event].V();
    }
}
class HEventer extends Thread {
    HighFrequencyCounter hfc;
    HEventer(HighFrequencyCounter hfc) {
        this.hfc = hfc;
    }
    public void run() {
        Random r = new Random();
        for (int i = 0; i < 20000; i++) {
            hfc.count(r.nextInt(10));
        }
    }
}
class TestHighFrequencyCounter {
    public static void main(String[] args) {
        final int E = 1000; // number of eventers
        HighFrequencyCounter hfc = new HighFrequencyCounter(10);
        HEventer hev[] = new HEventer[E];
        for (int i = 0; i < E; i++) hev[i] = new HEventer(hfc);
        long hstartTime = System.currentTimeMillis();
        for (int i = 0; i < E; i++) hev[i].start();
        for (int i = 0; i < E; i++) {
            try {hev[i].join();} catch (Exception e) {}
        }
        long hendTime = System.currentTimeMillis();
        System.out.println(hendTime - hstartTime);
    }
}

Overwriting highfrequencycounter.java


In [19]:
!javac highfrequencycounter.java

In [20]:
!java TestHighFrequencyCounter

5508


Measure the efficiency of your two implementations with the above test code; it creates 1,000 threads calling `count` 20,000 times each. What can you say about the efficiency of `FrequencyCounter` vs. `HighFrequencyCounter`?

Depending on the computer, the timing results may vastly differ. Servers running virtualization may spread threads over cores of multiple processors, slowing down access to shared memory as processor caches have to be updated. You will likely get plausible results when running on your own computer and _not_ on a Jupyter server. Give the processor model, number of cores, and processor frequency!  These found with `wmic cpu get name, max clock speed, numberofcores` on Windows, `lscpu` on Linux, and `sysctl -a machdep.cpu` on macOS and Linux.

cpu name: Intel(R) Core(TM) i5-10400F CPU @ 2.90GHz
max clock speed: 2904
number of cores: 6

from the results i noted that FC was more effecient than HFC. I conluded this was because of the extra computation of creating the semaphores for HFC and checking each time if a semaphore is available before adding a count to an event. FC was faster since it just had to lock the list, add an event and move on.